<a href="https://colab.research.google.com/github/fatinirq/AI/blob/main/LSTMSpoiler_Updated_1_Dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
#!pip install cloud-tpu-client==0.10 torch==1.12.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import torch_xla
#import torch_xla.core.xla_model as xm
#device = xm.xla_device()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Creates a linear module
fc = torch.nn.Linear(5, 2, bias=True)

# Copies the module to the XLA device (the first Cloud TPU core)
fc = fc.to(device)

# Creates a random feature tensor
features = torch.randn(3, 5, device=device, requires_grad=True)

# Runs and prints the module
output = fc(features)
print(output)

tensor([[ 0.4768,  0.5363],
        [-0.7206, -0.6558],
        [-0.5493, -0.9175]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
pathData="/content/drive/My Drive/SpoilerData/GoodRead/"
pathResutlts="/content/drive/My Drive/SpoilerData/GoodRead/Results/"
pathModel="/content/drive/My Drive/SpoilerData/GoodRead/Model/"

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 182 kB 104.6 MB/s 
     |████████████████████████████████| 7.6 MB 81.0 MB/s 


In [ ]:

print(torch.__version__)
lstm = nn.LSTM(3, 3)
lstm.weight_ih_l0

1.12.1+cu113


Parameter containing:
tensor([[ 0.1482,  0.3221,  0.5559],
        [-0.3872, -0.3388, -0.0269],
        [ 0.1899, -0.2591,  0.0012],
        [-0.1415,  0.5725, -0.1551],
        [ 0.1234,  0.1026, -0.5254],
        [-0.3078, -0.3254,  0.5738],
        [-0.0058, -0.3624,  0.1766],
        [ 0.0603, -0.3918, -0.2365],
        [ 0.2572,  0.3192, -0.0309],
        [ 0.5547, -0.1231,  0.2470],
        [-0.3423, -0.0052,  0.3116],
        [-0.2119,  0.1337, -0.3339]], requires_grad=True)

In [ ]:
a=torch.tensor([1,2,3])
b=torch.tensor([1,3,5])
c=a*b
c

tensor([ 1,  6, 15])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
#import spacy
import pandas as pd
from torchtext import data
#from torchtext.legacy.data import TabularDataset
from sklearn.model_selection import train_test_split
#from updatedRNN_model import RNN
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
#dataloader = DataLoader(iter(dfTrain['discourse_effectiveness',dfTrain['Preprocessed']), batch_size=8, shuffle=False, collate_fn=collate_batch)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SpoilerData/intensity.csv')


In [ ]:
# df=df[:100000]

In [ ]:
lst=df['topics'][0].replace("'",'').replace("[",'').replace(']','').split()
lst2=[float(item) for item in lst]
lst2.sort(reverse=True)
#lst2

In [ ]:
listind=[]
count=0
for ind in df.index:
    if len(df['review_sentences'][ind].strip())==0:
        listind.append(ind)

display(df.iloc[listind[0]])
if (len(listind)>0):
  df.drop(listind, axis=0, inplace=True)
display(df.iloc[listind[0]])

Unnamed: 0                                                       1136
review_sentences                                                     
user_id                              34216df2a5f3846b17e3f96bea6c2ad7
timestamp                                                  2017-08-15
rating                                                              3
has_spoiler                                                     False
book_id                                                       7818684
review_id                            ee2b6a6ef25de1ddf311faedfee82357
topics              [0.06873614 0.04656319 0.09090909 0.02439024 0...
log                                                               0.0
Name: 1136, dtype: object

Unnamed: 0                                                       1137
review_sentences    love harry potter story course would like thin...
user_id                              34216df2a5f3846b17e3f96bea6c2ad7
timestamp                                                  2016-05-22
rating                                                              5
has_spoiler                                                     False
book_id                                                      23734628
review_id                            d82af813eb4698f492eb2c3f9be644fb
topics              [0.10755149 0.0389016  0.09229596 0.00839054 0...
log                                                       -887.747986
Name: 1137, dtype: object

In [ ]:
_counter=0
for ind in df.index:
    if len(df['review_sentences'][ind].strip())>2000:
        _counter+=1
_counter

77878

In [ ]:
df.reset_index(drop=True)

,Unnamed: 0,review_sentences,user_id,timestamp,rating,has_spoiler,book_id,review_id,topics,log
0,0,also high energy cosmic ray entering atmospher...,8842281e1d1347389f2ab93d60773d4d,2017-08-30,5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,[0.10335387 0.02806297 0.04859685 0.02806297 0...,-1051.669067
1,1,avail free december http www audible com mt el...,8842281e1d1347389f2ab93d60773d4d,2017-03-22,3,False,16981,a5d2c3628987712d0e05c4f90798eb67,[0.10335387 0.02806297 0.04859685 0.02806297 0...,0.000000
2,2,cheat system live choice learn daniela say lif...,8842281e1d1347389f2ab93d60773d4d,2017-03-20,3,True,28684704,2ede853b14dc4583f96cf5d120af636f,[0.05639308 0.09547739 0.02847571 0.03405918 0...,-1197.603760
3,3,http www npr org recommended reading understan...,8842281e1d1347389f2ab93d60773d4d,2016-11-09,0,False,27161156,ced5675e55cd9d38a524743f5c40996e,[0.18128654 0.00584795 0.00584795 0.00584795 0...,-202.208206
4,4,surfer love end freya finding meaning surfing ...,8842281e1d1347389f2ab93d60773d4d,2016-04-25,4,True,25884323,332732725863131279a8e345b63ac33e,[0.07287295 0.03090424 0.06524228 0.03090424 0...,-1810.500488
...,...,...,...,...,...,...,...,...,...,...
1377686,1378028,travis abby travis abby wait travis pov,35cef391b171b4fca45771e508028212,2013-04-16,0,False,15745950,0e1db3d4b04256f9660f5d276ddf1314,[0.08183633 0.00199601 0.06187625 0.04191617 0...,0.000000
1377687,1378029,wait update finished read shelf forever,35cef391b171b4fca45771e508028212,2012-12-28,0,False,10861195,0b7f352e58caf0fd1f961e98ef04e89c,[0.08183633 0.00199601 0.06187625 0.04191617 0...,0.000000
1377688,1378030,regardless april cannot come soon enough ok ma...,35cef391b171b4fca45771e508028212,2013-03-25,4,False,6131164,9b19eff33ddb14e9e68fca2e90379e46,[0.10224439 0.07730673 0.00249377 0.00249377 0...,-355.610931
1377689,1378031,clockwork prince spell binding tale hooked sta...,35cef391b171b4fca45771e508028212,2013-01-24,4,False,10025305,8be463fed78f0da63e964706f710332b,[0.00126422 0.10240203 0.00126422 0.00126422 0...,-614.188171


In [ ]:
df.head()

,Unnamed: 0,review_sentences,user_id,timestamp,rating,has_spoiler,book_id,review_id,topics,log
0,0,also high energy cosmic ray entering atmospher...,8842281e1d1347389f2ab93d60773d4d,2017-08-30,5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,[0.10335387 0.02806297 0.04859685 0.02806297 0...,-1051.669067
1,1,avail free december http www audible com mt el...,8842281e1d1347389f2ab93d60773d4d,2017-03-22,3,False,16981,a5d2c3628987712d0e05c4f90798eb67,[0.10335387 0.02806297 0.04859685 0.02806297 0...,0.000000
2,2,cheat system live choice learn daniela say lif...,8842281e1d1347389f2ab93d60773d4d,2017-03-20,3,True,28684704,2ede853b14dc4583f96cf5d120af636f,[0.05639308 0.09547739 0.02847571 0.03405918 0...,-1197.603760
3,3,http www npr org recommended reading understan...,8842281e1d1347389f2ab93d60773d4d,2016-11-09,0,False,27161156,ced5675e55cd9d38a524743f5c40996e,[0.18128654 0.00584795 0.00584795 0.00584795 0...,-202.208206
4,4,surfer love end freya finding meaning surfing ...,8842281e1d1347389f2ab93d60773d4d,2016-04-25,4,True,25884323,332732725863131279a8e345b63ac33e,[0.07287295 0.03090424 0.06524228 0.03090424 0...,-1810.500488


In [ ]:
df.index=range(len(df))

In [ ]:
z= [len(x) for x in list(df['review_sentences'])]
z.sort(reverse=True)
_max=max(z)
_max=2000


In [ ]:
print(len(listind))

342


In [ ]:
lstParameter1=[]
lstParameter2=[]
for i in df.index:
  lst=df['topics'][i].replace("'",'').replace("[",'').replace(']','').split()
  lst2=[float(item) for item in lst]
  lst2.sort(reverse=True)
  lstParameter1.append(lst2[0])
  lstParameter2.append(lst2[1])

In [ ]:
df['param1']=lstParameter1
df['param2']=lstParameter2

In [ ]:
import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer("You can now install TorchText using pip!")
tokens

['you', 'can', 'now', 'install', 'torchtext', 'using', 'pip', '!']

In [ ]:
print(len(df.loc[((df['param1']+df['param2']) >= 0.2) & (df['has_spoiler']==True) ]))
print(len(df.loc[(df['has_spoiler']==True) ]))
print(len(df.loc[((df['param1']+df['param2']) >= 0.5) & (df['has_spoiler']==False) ]))

68940
89627
923


In [ ]:
vec = torchtext.vocab.GloVe(name='6B', dim=50)
#ret = vec.get_vecs_by_tokens(examples, lower_case_backup=True)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399999/400000 [00:08<00:00, 48811.06it/s]


In [ ]:
# _max=2000
# tokens=[tokenizer(item) for item in df['review_sentences']]
# lengths=[len(token) for token in tokens]
# vectors1=[vec.get_vecs_by_tokens(tokens[i][:_max], lower_case_backup=True) if lengths[i] > _max else vec.get_vecs_by_tokens(tokens[i]+['']*(_max-lengths[i]), lower_case_backup=True) for i in range(len(lengths))]

In [ ]:
print(len(lstParameter1))

1377691


In [ ]:
trainData,testData=train_test_split(df,test_size=0.2)

testData.reset_index(drop=True)#=range(len(testData))
testData.index=range(len(testData))
testData.head()
print(len(testData))
#del df

275539


In [ ]:
trainData.head()

,Unnamed: 0,review_sentences,user_id,timestamp,rating,has_spoiler,book_id,review_id,topics,log,param1,param2
579216,579378,love love love author wonderful storytelling s...,604f8693bfdcfdf6d01c3f5fd174f893,2016-11-17,5,False,18001399,9d4f8848e639e8174a47ce3b7e99cd96,[0.02439024 0.07084785 0.05923345 0.03600464 0...,-622.270996,0.082462,0.082462
220129,220224,vilma book blog facebook twitter review connec...,908b185d21d29d3dc12bbd414fee1ef2,2013-08-23,5,False,83144,d446c500af79a5c8b033bac031d03a47,[0.04927418 0.03700675 0.04314046 0.02473932 0...,-3041.244385,0.110611,0.065631
1358667,1359009,naturally given rise legendary fire hate heart...,df063ed2b0cd3ee53ca1e11a6ebcdcd0,2017-07-26,1,False,187181,dd1267b5f13b1d3658faf6babe39b12e,[0.11357341 0.08587258 0.05817175 0.03047092 0...,-318.057648,0.113573,0.085873
1237676,1237989,stayed several hour late finish prop issue inc...,063fa56483000312e88ea06c75cca06b,2017-07-27,4,False,28449257,1f595c0f7033b4bce513b970d5a2ee55,[0.08045977 0.04214559 0.04214559 0.00383142 0...,-265.565063,0.157088,0.080460
688259,688449,follow katniss peeta victory tour find despite...,c71b6632fcedf35444a931e6f2a8a8f7,2011-10-02,4,False,6148028,cf38fb2158a9d23681da2327e0284ade,[0.04564315 0.04564315 0.00414938 0.08713692 0...,-245.103424,0.170124,0.087137


In [ ]:
trainData,validData=train_test_split(trainData,test_size=0.2)

trainData.reset_index(drop=True)#=range(len(trainData))
trainData.index=range(len(trainData))
validData.reset_index(drop=True)#=range(len(validData))
validData.index=range(len(validData))

print(len(validData))
#del df

220431


In [ ]:
trainData.head()

,Unnamed: 0,review_sentences,user_id,timestamp,rating,has_spoiler,book_id,review_id,topics,log,param1,param2
0,898463,copy provided netgalley exchange honest review...,ed556b92506c3452b42fffed31697a1a,2015-07-19,5,False,23355896,1dfa237c478565d1e55b0def520d999e,[0.08500973 0.04607398 0.05256327 0.01362751 0...,-1050.955078,0.110967,0.104478
1,24784,good read always feel chest felt like expandin...,1b44140d6d78f09468bd27e20925029c,2015-10-09,5,False,1162022,51e93ed1e89873605fe8e70a39415281,[0.10747957 0.05091138 0.10747957 0.00691389 0...,-1003.280518,0.132621,0.107480
2,1098725,full review night owl review http www nightowl...,de8a5e808c457797cb1b357d8972e9d5,2016-06-11,3,False,216363,e962f521f803539bb8e04dad6b02a1d5,[0.02240326 0.08350305 0.08350305 0.00203666 0...,-400.378479,0.164969,0.144603
3,113928,ago remember really enjoying,d27de0320b0c8ebd47c811114afbef22,2014-03-26,3,False,47970,a851708ebe29b7524ce067dced941f8b,[0.03870162 0.03870162 0.05118602 0.05118602 0...,0.000000,0.176030,0.113608
4,961276,also sobbing chapter ten heart enough felt lik...,716cfad9b0f39fa8e15b4a9f5961859b,2016-04-01,4,False,37435,5bbe691fe5e914c5bc02d3eea9ed98a5,[0.04146839 0.05506458 0.11624745 0.014276 0...,-1059.143921,0.123046,0.116247


In [ ]:
trainData['param1'][0]


0.11096691

In [ ]:
validData["review_sentences"][1599]

'ending completely took surprise writing style refreshing drawing reader without resorting overly simplistic style may common book type pleasantly surprised complexity plotline something come expect typical teenage paranormal series shadow kiss rose must choose priority test strength struggle balance deepening bond lissa intensifying series strigoi attack blossoming forbidden relationship dimitri trainer learn shadow kiss book rose also see spirit although character initially unsure whether functional gift symptom madness rose shadow kissed faced death healed spiritually bound lissa healer way embed friend feeling interaction make ideal protector lissa vampire moroi one good albeit weaker breed series center around heroine rose fated guard best friend lissa danger strigoi evil breed immortal vampire set kill destroy many teen novel found vampire diary series whim previously knowing anything character plotline story something admit frequently indulge world teenage book series rarely exp

In [ ]:
torch.backends.cudnn.deterministic=True

In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, text, labels,param1,param2,_max):
        self.labels = labels
        self.text = text
        self.max=_max
        self.param1=torch.tensor(param1,dtype=torch.float32)
        self.param2=torch.tensor(param2,dtype=torch.float32)
        #self.param1=param1
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = tokenizer(self.text[idx])
        length=torch.tensor(len(text),dtype=torch.int64)
        text= vec.get_vecs_by_tokens(text[:_max], lower_case_backup=True) if length > self.max else vec.get_vecs_by_tokens(text+['']*(self.max-length), lower_case_backup=True)
        #print (length.shape)
        param1=self.param1[idx]
        param2=self.param2[idx]
        #text=vec.get_vecs_by_tokens(text, lower_case_backup=True)
        #print(text)
        #idxTemp= tokenizer(self.text[idx], truncation=True)['input_ids']
        #param1=self.param1
        sample = {"Text": text, "Class": label,'Param1':param1,'Param2':param2,'Length':length}#,"Vec":vector}
        return sample


In [ ]:
label_pipeline = lambda x: torch.tensor(x,dtype=torch.float32)

In [ ]:
batch_size=512

In [ ]:
# def collate_batch(batch):
#     _max=2000
#     label_list, text_list, lengths = [], [], []
#     text_tensor=torch.zeros([batch_size, _max,50], dtype=torch.float32)
#     label_tensor=torch.zeros([batch_size], dtype=torch.float32)
#     length_tensor= torch.zeros([batch_size], dtype=torch.int64)
#     for idx,item in  enumerate(batch):

#       _label=item['Class']
#       _text=item['Text']

#       label_list.append(label_pipeline(_label))
#       _length=len(_text)
#       if _length >_max:
#         _text=_text[:_max]
#         _length=_max
#       else:
#         _text=_text+['']*(_max-_length)
#       processed_text =  vec.get_vecs_by_tokens(_text, lower_case_backup=True)
#       #print(processed_text.shape)
#       text_list.append(processed_text)

#       text_tensor[idx]=processed_text
#       #label_tensor[idx]=label_pipeline(_label)
#       #length_tensor[idx]=_length
#       lengths.append(_length)
#     label_list = torch.tensor(label_list, dtype=torch.float32)
#     #lengths = torch.tensor(lengths[:-1]).cumsum(dim=0)
#     lengths = torch.tensor(lengths,dtype=torch.int64)
#     text_list = torch.stack(text_list)

#     return label_list.to(device), text_list.to(device), lengths.to(device)


In [ ]:
# def collate_batch2(batch):
#     _max=2000
#     label_list, text_list, lengths ,param1_list= [], [], [], []
#     text_tensor=torch.zeros([batch_size, _max,50], dtype=torch.float32)
#     label_tensor=torch.zeros([batch_size], dtype=torch.float32)
#     length_tensor= torch.zeros([batch_size], dtype=torch.int64)
#     for idx,item in  enumerate(batch):

#       _label=item['Class']
#       _text=item['Text']
#       _param1=torch.tensor(item['Param1'])
#       _param2=torch.tensor(item['Param2'])
#       #print(idx,'param1',_param1.shape)

#       label_list.append(label_pipeline(_label))
#       _length=len(_text)
#       if _length >_max:
#         _text=_text[:_max]
#         _length=_max
#       else:
#         _text=_text+['']*(_max-_length)
#       processed_text =  vec.get_vecs_by_tokens(_text, lower_case_backup=True) * (_param1+_param2)
#       #print(processed_text.shape)
#       text_list.append(processed_text)
#       #print(torch.tensor([item['Param1']+item['Param2']], dtype=torch.float32))
#       text_tensor[idx]=processed_text.to(device)
#       #label_tensor[idx]=label_pipeline(_label)
#       #length_tensor[idx]=_length
#       lengths.append(_length)
#       param1_list.append(_param1)
#     label_list = torch.tensor(label_list, dtype=torch.float32)
#     #lengths = torch.tensor(lengths[:-1]).cumsum(dim=0)
#     lengths = torch.tensor(lengths,dtype=torch.int64)
#     text_list = torch.stack(text_list)
#     #param1_list=torch.tensor(param1_list)
#     return label_list.to(device), text_list.to(device), lengths.to(device)


In [ ]:
def collate_batch3(batch):
    _max=2000
    #label_list, text_list, lengths ,param_list= [], [], [], []
    #text_tensor=torch.zeros([batch_size, _max,50], dtype=torch.float32)
    #label_tensor=torch.zeros([batch_size], dtype=torch.float32)
    #length_tensor= torch.zeros([batch_size], dtype=torch.int64)
    # embeddings = torch.stack([item['Text'][:_max]
    #                           if item['Length'] > _max
    #                           else vec.get_vecs_by_tokens(item['Text']+['']*(_max-item['Length']), lower_case_backup=True)
    embeddings = torch.stack([item['Text']  for item in batch])
    labels=torch.stack([label_pipeline(item['Class']) for item in batch])
    params=torch.stack([(item['Param1'].clone().detach()+item['Param2'].clone().detach()) *2 for item in batch])
    lengths= torch.stack([item['Length'] for item in batch])
    return labels, embeddings,lengths,params
    # for idx,item in  enumerate(batch):


    #   _label=item['Class']
    #   _text=item['Text']
    #   _param1=item['Param1'].clone().detach().requires_grad_(True)
    #   _param2=item['Param2'].clone().detach().requires_grad_(True)
    #   #print(idx,'param1',_param1.shape)

    #   label_list.append(label_pipeline(_label))
    #   _length=len(_text)
    #   if _length >_max:
    #     _text=_text[:_max]
    #     _length=_max
    #   else:
    #     _text=_text+['']*(_max-_length)
    #   processed_text =  vec.get_vecs_by_tokens(_text, lower_case_backup=True)
    #   #print(processed_text.shape)
    #   text_list.append(processed_text)
    #   param_list.append(_param1+_param2)
    #   #print(torch.tensor([item['Param1']+item['Param2']], dtype=torch.float32))
    #   #text_tensor[idx]=processed_text.to(device)
    #   #label_tensor[idx]=label_pipeline(_label)
    #   #length_tensor[idx]=_length
    #   lengths.append(_length)
    # label_list = torch.tensor(label_list, dtype=torch.float32)
    # #lengths = torch.tensor(lengths[:-1]).cumsum(dim=0)
    # lengths = torch.tensor(lengths,dtype=torch.int64)
    # text_list = torch.stack(text_list)
    # param_list=torch.tensor(param_list,dtype=torch.float32)
    # return label_list, text_list, lengths,param_list


In [ ]:
# define data and class labels
##text = ['Happy', 'Amazing', 'Sad', 'Unhapy', 'Glum']
##labels = ['Positive', 'Positive', 'Negative', 'Negative', 'Negative']
# create Pandas DataFrame
##text_labels_df = pd.DataFrame({'Text': text, 'Labels': labels})
# define data set object
_max=2000

TD = CustomTextDataset(trainData['review_sentences'],trainData['has_spoiler'],trainData['param1'],trainData['param2'],_max)
TTD=CustomTextDataset(testData['review_sentences'],testData['has_spoiler'],testData['param1'],testData['param2'],_max)
VD=CustomTextDataset(validData['review_sentences'],validData['has_spoiler'],validData['param1'],validData['param2'],_max)

In [ ]:
# trainIter = DataLoader(TD, batch_size=batch_size, shuffle=True,collate_fn=collate_batch)
# testIter = DataLoader(TTD, batch_size=batch_size, shuffle=True,collate_fn=collate_batch)
# validIter = DataLoader(VD, batch_size=batch_size, shuffle=True,collate_fn=collate_batch)

In [ ]:
# trainIter2 = DataLoader(TD, batch_size=batch_size, shuffle=True,collate_fn=collate_batch2)
# testIter2 = DataLoader(TTD, batch_size=batch_size, shuffle=True,collate_fn=collate_batch2)
# validIter2 = DataLoader(VD, batch_size=batch_size, shuffle=True,collate_fn=collate_batch2)

In [ ]:
print(torch.tensor(df['param2'][3]))

tensor(0.0643, dtype=torch.float64)


In [ ]:
trainIter3 = DataLoader(TD, batch_size=batch_size,collate_fn=collate_batch3,num_workers=4,prefetch_factor=8,pin_memory=True,shuffle=True)
testIter3 = DataLoader(TTD, batch_size=batch_size,collate_fn=collate_batch3,num_workers=4,prefetch_factor=8,pin_memory=True,shuffle=True)
validIter3 = DataLoader(VD, batch_size=batch_size,collate_fn=collate_batch3,num_workers=4,prefetch_factor=8,pin_memory=True,shuffle=True)

In [ ]:
# for idx, batch in enumerate(TD):
#   print(batch['Class'])
#   if idx>2:
#     break
for idx, (label, text,lengths,params) in enumerate(trainIter3):
    # Print the 'text' data of the batch
    print(idx)
    print("______________")
    print("Label=",label.shape)
    print("Text=",text.shape)
    print("offsets=",lengths)
    print("params=",params.shape)
    if idx>1:
      break

0
______________
Label= torch.Size([512])
Text= torch.Size([512, 2000, 50])
offsets= tensor([108,  24, 129,   8,  22,  16,  57, 157, 218,  73,  57,  30, 190,  67,
        106, 231,  36, 202, 138, 635,   9,  27,   6,  66, 384, 112, 240, 183,
         30,  29,   4, 192, 183,  18,  11,   9,  12,  23,  41, 331, 215, 101,
          4, 294, 301,  28, 273,  17, 220, 379,  22,  70,   7,   2, 298,  72,
          8,  17, 144, 278,  45,  20,  12,  21,  27, 131,  12,  13,   7,  65,
          3,  30,  33,  11,   5, 130,  64, 100,  69,  72,  86,  28,  25,   7,
         45,  13,   4,  20,  84,  71,  15,  95,   8,  58, 124, 294,  45, 250,
        154, 117, 270, 126,  16,  19,   2,   5,   4,  82,   2,  34,  94,   8,
         44, 136, 403,  64, 163,  38,   7,  32,   5, 204,  83, 572,  84, 350,
         96,  55,  36,  54,  68, 179, 152, 141,  66, 135,  88, 328, 365,  34,
         79, 158,  39, 100, 102, 267, 247,  10,  57,  16, 252,   6,  55,  95,
         71,  51,  86,  42, 185, 491, 105, 158,  75,   3,

In [ ]:
trainData['review_sentences'][1136]

'grade c nice action fright mention least page long thing character narrative thrust fun stephen king took realistic situation superflu wiping world put supernatural force top despite derivative stephen king classic stand take element make masterpiece ugh transform bat collective mind oh yeah also supernatural little girl talk via telepathy human animal hope super strength stand light thirst human blood live forever something like vampire actual vampire within first page suppose accept supervirus creating vampire passage suspend disbelief review game throne willing accept fictional world kingdom mysterious zombie northern land fiction tricky '

In [ ]:
import gc
gc.enable()
gc.collect()

0

In [ ]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional,
                 dropout_rate,pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim,sparse=True,padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional=bidirectional,
                            dropout=dropout_rate, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, text, offsets):
        # ids = [batch size, seq len]
        # length = [batch size]
        #rint(text.shape)
        #embedded = self.embedding(text)
        # embedded = [batch size, seq len, embedding dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(text, offsets.type(torch.int64).cpu(), batch_first=True, enforce_sorted=False)

        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        output, output_length = nn.utils.rnn.pad_packed_sequence(packed_output)
        # output = [batch size, seq len, hidden dim * n directions]
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat([hidden[-1], hidden[-2]], dim=-1))
            # hidden = [batch size, hidden dim * 2]
        else:
            hidden = self.dropout(hidden[-1])
            # hidden = [batch size, hidden dim]
        prediction = self.fc(hidden)
        # prediction = [batch size, output dim]
        return prediction


In [ ]:
class LSTM3(nn.Module):
    def __init__(self,  features_dim, hidden_dim, output_dim, n_layers, bidirectional,
                 dropout_rate):
        super().__init__()
        #self.embedding = nn.Embedding(vocab_size, embedding_dim,sparse=True,padding_idx=pad_idx)
        self.lstm = nn.LSTM(features_dim, hidden_dim, n_layers, bidirectional=bidirectional,
                            dropout=dropout_rate, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.fc2 = nn.Linear(output_dim+1, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, text, offsets,param):
        # ids = [batch size, seq len]
        # length = [batch size]
        #rint(text.shape)
        #embedded = self.embedding(text)
        # embedded = [batch size, seq len, embedding dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(text, offsets.type(torch.int64).cpu(), batch_first=True, enforce_sorted=False)

        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        output, output_length = nn.utils.rnn.pad_packed_sequence(packed_output)
        # output = [batch size, seq len, hidden dim * n directions]
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat([hidden[-1], hidden[-2]], dim=-1))
            # hidden = [batch size, hidden dim * 2]
        else:
            hidden = self.dropout(hidden[-1])
            # hidden = [batch size, hidden dim]
        temPrediction = self.fc1(hidden)
        #print(param.get_device())
        #print(temPrediction.get_device())
        prediction=self.fc2(torch.stack((param,temPrediction.squeeze()),dim=-1))
        # prediction = [batch size, output dim]
        return prediction


In [ ]:
from torchtext.vocab import GloVe, vocab
unk_token = "pad"
pad_token="unk"

glove_vectors = vec
glove_vocab = vocab(glove_vectors.stoi)
unk_index = glove_vectors.stoi[unk_token]

#this is necessary otherwise it will throw runtime error if OOV token is queried
glove_vocab.set_default_index(unk_index)
pretrained_embeddings = glove_vectors.vectors
pretrained_embeddings = torch.cat((torch.zeros(1,pretrained_embeddings.shape[1]),pretrained_embeddings))
UNK_IDX = glove_vectors.stoi[unk_token]
PAD_IDX = glove_vectors.stoi[pad_token]

In [ ]:
INPUT_DIM = len(glove_vectors.vectors)
print(INPUT_DIM)
EMBEDDING_DIM = 50
HIDDEN_DIM = 100
OUTPUT_DIM = 1
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = glove_vectors.stoi[pad_token]


400000


In [ ]:
model = LSTM(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,PAD_IDX)
print(model)


model3 = LSTM3(
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT)
print(model3)

LSTM(
  (embedding): Embedding(400000, 50, padding_idx=201534, sparse=True)
  (lstm): LSTM(50, 100, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=200, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
LSTM3(
  (lstm): LSTM(50, 100, batch_first=True, dropout=0.5, bidirectional=True)
  (fc1): Linear(in_features=200, out_features=1, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


print(model.embedding.weight.data)

tensor([[-2.0184, -0.5705, -1.6740,  ..., -0.2228,  0.8013, -1.0592],
        [ 1.4134, -0.1229,  1.0986,  ..., -0.0856, -1.2477,  0.1680],
        [ 1.2120,  0.3198, -0.7829,  ..., -0.3074,  0.5359,  0.1375],
        ...,
        [ 1.9423,  0.7974,  0.7858,  ...,  2.2333, -0.2870,  0.8297],
        [-0.8613, -1.3938,  0.5874,  ...,  1.2875,  0.0369,  1.9340],
        [-0.4313,  0.2161,  0.0671,  ...,  1.2551,  1.2085, -0.2148]])


In [ ]:
# model3.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
# model3.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
# print(model3.embedding.weight.data)

In [ ]:
!pip install pytorch-model-summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,121,801 trainable parameters


In [ ]:
print(f'The model has {count_parameters(model3):,} trainable parameters')

The model has 121,804 trainable parameters


In [ ]:
torch.cuda.is_available()

True

In [ ]:
import torch.optim as optim

print(torch.__version__)
optimizer = optim.Adam(model.parameters())
optimizer3=optim.Adam(model3.parameters())

criterion =nn.BCEWithLogitsLoss()

criterion = criterion.to(device)
model = model.to(device)
model3=model3.to(device)


1.12.1+cu113


In [ ]:
import numpy as np
from sklearn import metrics
y = np.array([1, 1, 2, 2])
pred = np.array([0.1, 0.4, 0.35, 0.8])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
metrics.auc(fpr, tpr)


0.75

In [ ]:
def focal_loss(bce_loss, targets, gamma, alpha):
    """Binary focal loss, mean.

    Per https://discuss.pytorch.org/t/is-this-a-correct-implementation-for-focal-loss-in-pytorch/43327/5 with
    improvements for alpha.
    :param bce_loss: Binary Cross Entropy loss, a torch tensor.
    :param targets: a torch tensor containing the ground truth, 0s and 1s.
    :param gamma: focal loss power parameter, a float scalar.
    :param alpha: weight of the class indicated by 1, a float scalar.
    """
    p_t = torch.exp(-bce_loss)
    alpha_tensor = (1 - alpha) + targets * (2 * alpha - 1)  # alpha if target = 1 and 1 - alpha if target = 0
    f_loss = alpha_tensor * (1 - p_t) ** gamma * bce_loss
    return f_loss.mean()

In [ ]:
def binary_accuracy(rounded_preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer

    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(rounded_preds)
    return acc

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 5.0 MB/s 


In [ ]:
import torchmetrics
from torchmetrics.classification import PrecisionRecallCurve
#from torchmetrics import AUC



# initialize metric

def train(model, iterator, optimizer, criterion):
    #aucMetric =0 # AUC(compute_on_step=True,reorder=True)
    F1Metric=torchmetrics.F1Score(task='binary',compute_on_step=True).cuda()

    PrecMetric=torchmetrics.Precision(task='binary',compute_on_step=True).cuda()
    RecallMetric=torchmetrics.Recall(task='binary',compute_on_step=True).cuda()
    aucRCurveMetric=torchmetrics.AUROC(task='binary',compute_on_step=True).cuda()
    epoch_loss = 0
    epoch_acc = 0
    epoch_auc=0
    TP_train,TN_train,FP_train,FN_train=0,0,0,0
    model.train()
    gamma = torch.tensor(1.0, dtype = torch.float32).cuda()
    alpha=torch.tensor(0.9, dtype = torch.float32).cuda()
    for idx, (label, text, offsets) in enumerate(iterator):

        optimizer.zero_grad()

        #print(batch)
        #print(batch.data)

        predictions=model(text,offsets).squeeze(1)

        predictionsigmoid=torch.sigmoid(predictions)
        rounded_preds=torch.round(predictionsigmoid)
        #if idx==0:
        #  print('predictions',predictionsigmoid)

        #print(rounded_preds)

        #print("Batch labels  "+ batch.label.int().cuda())
        bce_loss = criterion(predictions, label)
        loss= focal_loss(bce_loss,label,gamma,alpha)
        #print(loss)

        acc = binary_accuracy(rounded_preds, label)
        #aucMetric.update(rounded_preds, label)
        F1Metric.update(rounded_preds, label.type(torch.IntTensor).cuda())
        PrecMetric.update(rounded_preds, label.type(torch.IntTensor).cuda())
        RecallMetric.update(rounded_preds,label.type(torch.IntTensor).cuda())
        aucRCurveMetric.update(rounded_preds, label.type(torch.IntTensor).cuda())
        for i in range(len(label)):
          if label[i]==0 and rounded_preds[i]==0:
            TN_train+=1
           # print (TN_train)
          elif label[i]==1 and rounded_preds[i]==1:
            TP_train+=1
          elif label[i]==1 and rounded_preds[i]==0:
            FP_train+=1
          elif label[i]==0 and rounded_preds[i]==1:
            FN_train+=1

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    allMetrics={'AUC':0,'Precision':PrecMetric.compute(),'Recall':RecallMetric.compute(),
                'F1':F1Metric.compute(),'AUCR':aucRCurveMetric.compute()}

    #allMetrics={'AUC':aucMetric.compute(),'F1':F1Metric.compute(),'Precision':PrecMetric.compute(),
    #            'Recall':RecallMetric.compute(),'AUCR':aucRCurveMetric.compute()}

    return epoch_loss / len(iterator), epoch_acc / len(iterator),allMetrics,TP_train,TN_train,FP_train,FN_train

In [ ]:
from torchmetrics.classification import PrecisionRecallCurve

#from torchmetrics import AUC
import torchmetrics


# initialize metric

def train3(model, iterator, optimizer, criterion):
    #aucMetric =0
    F1Metric=torchmetrics.F1Score(task='binary',compute_on_step=True).cuda()

    PrecMetric=torchmetrics.Precision(task='binary',compute_on_step=True).cuda()
    RecallMetric=torchmetrics.Recall(task='binary',compute_on_step=True).cuda()
    aucRCurveMetric=torchmetrics.AUROC(task='binary',compute_on_step=True).cuda()
    epoch_loss = 0
    epoch_acc = 0
    epoch_auc=0
    TP_train,TN_train,FP_train,FN_train=0,0,0,0
    model.train()
    gamma = torch.tensor(1.0, dtype = torch.float32).cuda()
    alpha=torch.tensor(0.9, dtype = torch.float32).cuda()
    for idx, (label, text, offsets,param) in enumerate(iterator):

        optimizer.zero_grad()

        #print(batch)
        #print(batch.data)

        predictions=model(text.to(device),offsets.to(device),param.to(device).requires_grad()).squeeze(1)

        predictionsigmoid=torch.sigmoid(predictions)
        rounded_preds=torch.round(predictionsigmoid)
        #if idx==0:
        #  print('predictions',predictionsigmoid)

        #print(rounded_preds)

        #print("Batch labels  "+ batch.label.int().cuda())
        bce_loss = criterion(predictions, label.to(device))
        loss= focal_loss(bce_loss,label.to(device),gamma,alpha)
        #print(loss)

        acc = binary_accuracy(rounded_preds, label.to(device))
        #aucMetric.update(rounded_preds, label)
        F1Metric.update(rounded_preds, label.type(torch.IntTensor).cuda())
        PrecMetric.update(rounded_preds, label.type(torch.IntTensor).cuda())
        RecallMetric.update(rounded_preds,label.type(torch.IntTensor).cuda())
        aucRCurveMetric.update(rounded_preds, label.type(torch.IntTensor).cuda())
        for i in range(len(label)):
          if label[i]==0 and rounded_preds[i]==0:
            TN_train+=1
           # print (TN_train)
          elif label[i]==1 and rounded_preds[i]==1:
            TP_train+=1
          elif label[i]==1 and rounded_preds[i]==0:
            FP_train+=1
          elif label[i]==0 and rounded_preds[i]==1:
            FN_train+=1

        loss.backward()

        optimizer3.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    allMetrics={'AUC':0,'Precision':PrecMetric.compute(),'Recall':RecallMetric.compute(),
                'F1':F1Metric.compute(),'AUCR':aucRCurveMetric.compute()}

    #allMetrics={'AUC':aucMetric.compute(),'F1':F1Metric.compute(),'Precision':PrecMetric.compute(),
    #            'Recall':RecallMetric.compute(),'AUCR':aucRCurveMetric.compute()}

    return epoch_loss / len(iterator), epoch_acc / len(iterator),allMetrics,TP_train,TN_train,FP_train,FN_train

In [ ]:
def evaluate(model, iterator, criterion):


    #aucMetric = AUC(compute_on_step=True,reorder=True)
    F1Metric=torchmetrics.F1Score(task='binary',compute_on_step=True).cuda()

    PrecMetric=torchmetrics.Precision(task='binary',compute_on_step=True).cuda()
    RecallMetric=torchmetrics.Recall(task='binary',compute_on_step=True).cuda()
    aucRCurveMetric=torchmetrics.AUROC(task='binary',compute_on_step=True).cuda()
    epoch_loss = 0
    epoch_acc = 0
    epoch_auc=0

    gamma = torch.tensor(1.0, dtype = torch.float32).cuda()
    alpha=torch.tensor(0.5, dtype = torch.float32).cuda()
    model.eval()

    with torch.no_grad():

        for idx, (label, text, offsets) in enumerate(iterator):


            predictions = model(text,offsets).squeeze(1)


            predictionsigmoid=torch.sigmoid(predictions)
            rounded_preds=torch.round(predictionsigmoid)


        #print(rounded_preds)

        #print("Batch labels  "+ batch.label.int().cuda())
            bce_loss = criterion(predictions, label)
            loss= focal_loss(bce_loss,label,gamma,alpha)
        #print(loss)

            acc = binary_accuracy(rounded_preds, label)
            #aucMetric.update(rounded_preds, label)
            F1Metric.update(rounded_preds, label.type(torch.IntTensor).cuda())
            PrecMetric.update(rounded_preds, label.type(torch.IntTensor).cuda())
            RecallMetric.update(rounded_preds,label.type(torch.IntTensor).cuda())
            aucRCurveMetric.update(rounded_preds, label.type(torch.IntTensor).cuda())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    allMetrics={'AUC':0,'Precision':PrecMetric.compute(),'Recall':RecallMetric.compute(),
                'F1':F1Metric.compute(),'AUCR':aucRCurveMetric.compute()}

    #allMetrics={'AUC':aucMetric.compute(),'F1':F1Metric.compute(),'Precision':PrecMetric.compute(),
    #            'Recall':RecallMetric.compute(),'AUCR':aucRCurveMetric.compute()}

    return epoch_loss / len(iterator), epoch_acc / len(iterator),allMetrics,TP_train,TN_train,FP_train,FN_train



In [ ]:
def evaluate3(model, iterator, criterion):


    #aucMetric = 0
    F1Metric=torchmetrics.F1Score(task='binary',compute_on_step=True).cuda()

    PrecMetric=torchmetrics.Precision(task='binary',compute_on_step=True).cuda()
    RecallMetric=torchmetrics.Recall(task='binary',compute_on_step=True).cuda()
    aucRCurveMetric=torchmetrics.AUROC(task='binary',compute_on_step=True).cuda()
    epoch_loss = 0
    epoch_acc = 0
    epoch_auc=0
    TP_train,TN_train,FP_train,FN_train=0,0,0,0
    gamma = torch.tensor(1.0, dtype = torch.float32).cuda()
    alpha=torch.tensor(0.5, dtype = torch.float32).cuda()
    model3.eval()

    with torch.no_grad():

        for idx, (label, text, offsets, param) in enumerate(iterator):


            predictions = model3(text.to(device),offsets.to(device),param.to(device)).squeeze(1)


            predictionsigmoid=torch.sigmoid(predictions)
            rounded_preds=torch.round(predictionsigmoid)


        #print(rounded_preds)

        #print("Batch labels  "+ batch.label.int().cuda())
            bce_loss = criterion(predictions, label.to(device))
            loss= focal_loss(bce_loss,label.to(device),gamma,alpha)
        #print(loss)

            acc = binary_accuracy(rounded_preds, label.to(device))
            #aucMetric.update(rounded_preds, label)
            F1Metric.update(rounded_preds, label.type(torch.IntTensor).cuda())
            PrecMetric.update(rounded_preds, label.type(torch.IntTensor).cuda())
            RecallMetric.update(rounded_preds,label.type(torch.IntTensor).cuda())
            aucRCurveMetric.update(rounded_preds, label.type(torch.IntTensor).cuda())
            for i in range(len(label)):
                 if label[i]==0 and rounded_preds[i]==0:
                      TN_train+=1
           # print (TN_train)
                 elif label[i]==1 and rounded_preds[i]==1:
                      TP_train+=1
                 elif label[i]==1 and rounded_preds[i]==0:
                      FP_train+=1
                 elif label[i]==0 and rounded_preds[i]==1:
                      FN_train+=1

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    allMetrics={'AUC':0,'Precision':PrecMetric.compute(),'Recall':RecallMetric.compute(),
                'F1':F1Metric.compute(),'AUCR':aucRCurveMetric.compute()}

    #allMetrics={'AUC':aucMetric.compute(),'F1':F1Metric.compute(),'Precision':PrecMetric.compute(),
    #            'Recall':RecallMetric.compute(),'AUCR':aucRCurveMetric.compute()}

    return epoch_loss / len(iterator), epoch_acc / len(iterator),allMetrics,TP_train,TN_train,FP_train,FN_train

In [ ]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 4.7 MB/s 


In [ ]:
from datetime import datetime
import xlsxwriter
from xlrd import open_workbook
import xlwt
import os.path
filename=datetime.now().strftime("%d-%b-%Y")+".xlsx"


In [ ]:
os.environ['WANDB_CONSOLE'] = 'off'

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
sheetName=datetime.now().strftime("%H-%M-%S")
workbook = xlsxwriter.Workbook(sheetName)
N_EPOCHS = 10

best_valid_loss = float('inf')

dfData=pd.DataFrame()

Epoch=[]
listDfData=[]
train_lossList,valid_lossList,test_lossList=[],[],[]
train_accList,valid_accList,test_accList=[],[],[]
train_aucList,valid_aucList,test_aucList=[],[],[]
train_auCuList,valid_auCuList,test_auCuList=[],[],[]
train_F1List,valid_F1List,test_F1List=[],[],[]
train_PreList,valid_PreList,test_PreList=[],[],[]
train_RecallList,valid_RecallList,test_RecallList=[],[],[]
train_FPList,valid_FPList,test_FPList=[],[],[]
train_TPList,valid_TPList,test_TPList=[],[],[]
train_FNList,valid_FNList,test_FNList=[],[],[]
train_TNList,valid_TNList,test_TNList=[],[],[]

torch.cuda.empty_cache()
for epoch in tqdm(range(N_EPOCHS)):




    start_time = time.time()

    train_loss, train_acc,train_allMetrics,train_TP,train_TN,train_FP,train_FN = train3(model3, trainIter3, optimizer, criterion)
    valid_loss, valid_acc,valid_allMetrics,valid_TP,valid_TN,valid_FP,valid_FN = evaluate3(model3, validIter3, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model3.state_dict(), pathModel+'/'+datetime.now().strftime("%d-%b-%Y")+'tut1-model3.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f} | All Metrics:{train_allMetrics} | TP:{train_TP} | TN: {train_TN}')

    #print(f'\tRecall:{TP_train / (FN_train + TP_train):.3f} | Precision:{TP_train / (FP_train + TP_train):.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f} | Val All Metrics:{train_allMetrics}')
    model3.load_state_dict(torch.load(pathModel+'/'+datetime.now().strftime("%d-%b-%Y")+'tut1-model3.pt'))
    test_loss, test_acc,test_allMetrics,test_TP,test_TN,test_FP,test_FN= evaluate3(model3, testIter3, criterion)
    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f} | Test All Metrics:{train_allMetrics}')

    Epoch.append(epoch)
    train_lossList.append(train_loss)
    valid_lossList.append(valid_loss)
    test_lossList.append(test_loss)
    train_accList.append(train_acc*100)
    valid_accList.append(valid_acc*100)
    test_accList.append(test_acc*100)

    # train_aucList.append(train_allMetrics['AUC'].cpu().numpy())
    # valid_aucList.append(valid_allMetrics['AUC'].cpu().numpy())
    # test_aucList.append(test_allMetrics['AUC'].cpu().numpy())
    train_aucList.append(0)
    valid_aucList.append(0)
    test_aucList.append(0)
    train_auCuList.append(train_allMetrics['AUCR'].cpu().numpy())
    valid_auCuList.append(valid_allMetrics['AUCR'].cpu().numpy())
    test_auCuList.append(test_allMetrics['AUCR'].cpu().numpy())
    train_F1List.append(train_allMetrics['F1'].cpu().numpy())
    valid_F1List.append(valid_allMetrics['F1'].cpu().numpy())
    test_F1List.append(test_allMetrics['F1'].cpu().numpy())
    train_PreList.append(train_allMetrics['Precision'].cpu().numpy())
    valid_PreList.append(valid_allMetrics['Precision'].cpu().numpy())
    test_PreList.append(test_allMetrics['Precision'].cpu().numpy())
    train_RecallList.append(train_allMetrics['Recall'].cpu().numpy())
    valid_RecallList.append(valid_allMetrics['Recall'].cpu().numpy())
    test_RecallList.append(test_allMetrics['Recall'].cpu().numpy())
    train_FPList.append(train_FP)
    valid_FPList.append(valid_FP)
    test_FPList.append(test_FP)
    train_FNList.append(train_FN)
    valid_FNList.append(valid_FN)
    test_FNList.append(test_FN)
    train_FPList.append(train_TP)
    valid_FPList.append(valid_TP)
    test_FPList.append(test_TP)
    train_FNList.append(train_TN)
    valid_FNList.append(valid_TN)
    test_FNList.append(test_TN)

dfData.insert(0, 'Epoch', Epoch)
dfData.insert(1, 'train_loss_', train_lossList)
dfData.insert(2, 'valid_loss_', valid_lossList)
dfData.insert(3, 'test_loss_', test_lossList)
dfData.insert(4, 'train_acc_', train_accList)
dfData.insert(5, 'valid_acc_', valid_accList)
dfData.insert(6, 'test_acc_', test_accList)
dfData.insert(7, 'train_auc_', train_aucList)
dfData.insert(8, 'valid_auc_', valid_aucList)
dfData.insert(9, 'test_auc_', test_aucList)
dfData.insert(10, 'train_AUCR_', train_auCuList)
dfData.insert(11, 'valid_AUCR_', valid_auCuList)
dfData.insert(12, 'test_AUCR_', test_F1List)
dfData.insert(13, 'train_F1_', train_F1List)
dfData.insert(14, 'valid_F1_', valid_F1List)
dfData.insert(15, 'test_F1_', test_auCuList)
dfData.insert(16, 'train_Precision_', train_PreList)
dfData.insert(17, 'valid_Precision_', valid_PreList)
dfData.insert(18, 'test_Precision_', test_PreList)
dfData.insert(19, 'train_Recall_', train_RecallList)
dfData.insert(20, 'valid_Recall_', valid_RecallList)
dfData.insert(21, 'test_Recall_', test_RecallList)
dfData.insert(22, 'train_FP_', train_FPList)
dfData.insert(23, 'valid_FP_', valid_FPList)
dfData.insert(24, 'test_FP_', test_FPList)
dfData.insert(25, 'train_FN_', train_FNList)
dfData.insert(26, 'valid_FN_', valid_FNList)
dfData.insert(27, 'test_FN_', test_FNList)
dfData.insert(28, 'train_TP_', train_TPList)
dfData.insert(29, 'valid_TP_', valid_TPList)
dfData.insert(30, 'test_TP_', test_TPList)
dfData.insert(31, 'train_TN_', train_TNList)
dfData.insert(32, 'valid_TN_', valid_TNList)
dfData.insert(33, 'test_TN_', test_TNList)



  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 56m 35s
	Train Loss: 0.010 | Train Acc: 93.49 | All Metrics:{'AUC': 0, 'Precision': tensor(0., device='cuda:0'), 'Recall': tensor(0., device='cuda:0'), 'F1': tensor(0., device='cuda:0'), 'AUCR': tensor(0.5000, device='cuda:0')} | TP:0 | TN: 824383
	 Val. Loss: 0.028 |  Val. Acc: 93.50 | Val All Metrics:{'AUC': 0, 'Precision': tensor(0., device='cuda:0'), 'Recall': tensor(0., device='cuda:0'), 'F1': tensor(0., device='cuda:0'), 'AUCR': tensor(0.5000, device='cuda:0')}


 10%|█         | 1/10 [1:11:47<10:46:06, 4307.44s/it]

Test Loss: 0.028 | Test Acc: 93.43 | Test All Metrics:{'AUC': 0, 'Precision': tensor(0., device='cuda:0'), 'Recall': tensor(0., device='cuda:0'), 'F1': tensor(0., device='cuda:0'), 'AUCR': tensor(0.5000, device='cuda:0')}
Epoch: 02 | Epoch Time: 60m 55s
	Train Loss: 0.013 | Train Acc: 92.86 | All Metrics:{'AUC': 0, 'Precision': tensor(0.0710, device='cuda:0'), 'Recall': tensor(0.0082, device='cuda:0'), 'F1': tensor(0.0148, device='cuda:0'), 'AUCR': tensor(0.5004, device='cuda:0')} | TP:472 | TN: 818272
	 Val. Loss: 0.043 |  Val. Acc: 93.46 | Val All Metrics:{'AUC': 0, 'Precision': tensor(0.0710, device='cuda:0'), 'Recall': tensor(0.0082, device='cuda:0'), 'F1': tensor(0.0148, device='cuda:0'), 'AUCR': tensor(0.5004, device='cuda:0')}


 20%|██        | 2/10 [2:27:45<9:54:01, 4455.13s/it] 

Test Loss: 0.028 | Test Acc: 93.44 | Test All Metrics:{'AUC': 0, 'Precision': tensor(0.0710, device='cuda:0'), 'Recall': tensor(0.0082, device='cuda:0'), 'F1': tensor(0.0148, device='cuda:0'), 'AUCR': tensor(0.5004, device='cuda:0')}
Epoch: 03 | Epoch Time: 60m 28s
	Train Loss: 0.016 | Train Acc: 91.91 | All Metrics:{'AUC': 0, 'Precision': tensor(0.0662, device='cuda:0'), 'Recall': tensor(0.0188, device='cuda:0'), 'F1': tensor(0.0292, device='cuda:0'), 'AUCR': tensor(0.5002, device='cuda:0')} | TP:1074 | TN: 809289
	 Val. Loss: 0.055 |  Val. Acc: 93.46 | Val All Metrics:{'AUC': 0, 'Precision': tensor(0.0662, device='cuda:0'), 'Recall': tensor(0.0188, device='cuda:0'), 'F1': tensor(0.0292, device='cuda:0'), 'AUCR': tensor(0.5002, device='cuda:0')}


 20%|██        | 2/10 [3:28:51<13:55:24, 6265.53s/it]


KeyboardInterrupt: ignored

In [ ]:
from datetime import datetime
import xlsxwriter
from xlrd import open_workbook
import openpyxl
import xlwt
import os.path
filename='Model3'+datetime.now().strftime("%d-%b-%Y")+".xlsx"
sheetName=datetime.now().strftime("%H-%M-%S")
workbook = xlsxwriter.Workbook(pathResutlts + filename)
if os.path.isfile(pathResutlts+filename):
    print('old file')
    ExcelWorkbook = openpyxl.load_workbook(pathResutlts+filename)

    # Generating the writer engine
    writer = pd.ExcelWriter(pathResutlts+filename, engine = 'openpyxl')

    # Assigning the workbook to the writer engine
    writer.book = ExcelWorkbook

    dfData.to_excel(writer, sheet_name = sheetName)

    writer.save()
else:

    # saving the excel
    dfData.to_excel(pathResutlts+filename,sheet_name=sheetName)
    print('DataFrame is written to Excel File successfully.')


In [ ]:

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
writer = SummaryWriter('tensorboard/runs')
N_EPOCHS = 3


best_valid_loss = float('inf')

Epoch=[]
train_lossList,valid_lossList,test_lossList=[],[],[]
train_accList,valid_accList,test_accList=[],[],[]
train_aucList,valid_aucList,test_aucList=[],[],[]
torch.cuda.empty_cache()
for epoch in tqdm(range(N_EPOCHS)):




    start_time = time.time()

    train_loss, train_acc,train_allMetrics,TP_train,TN_train,FP_train,FN_train = train(model, trainIter, optimizer, criterion)
    valid_loss, valid_acc,valid_allMetrics = evaluate(model, validIter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f} | All Metrics:{train_allMetrics} | TP:{TP_train} | TN: {TN_train}')
    print(TN_train)

    print(f'\tRecall:{TP_train / (FN_train + TP_train):.3f} | Precision:{TP_train / (FP_train + TP_train):.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f} | Val All Metrics:{train_allMetrics}')
    model.load_state_dict(torch.load('tut1-model.pt'))
    test_loss, test_acc,test_allMetrics= evaluate(model, testIter, criterion)
    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f} | Test All Metrics:{train_allMetrics}')
    Epoch.append(epoch)
    train_lossList.append(train_loss)
    valid_lossList.append(valid_loss)
    test_lossList.append(test_loss)
    train_accList.append(train_acc*100)
    valid_accList.append(valid_acc*100)
    test_accList.append(test_acc*100)

    train_aucList.append(train_allMetrics)
    valid_aucList.append(valid_allMetrics)
    test_aucList.append(test_allMetrics)
    writer.add_scalar('Loss/train', round(train_loss,1), epoch)
    writer.add_scalar('Loss/valid', round(valid_loss,1), epoch)
    writer.add_scalar('Loss/test', round(test_loss,1), epoch)
    writer.add_scalar('Accuracy/train', int(train_acc*100), epoch)
    writer.add_scalar('Accuracy/valid',int(valid_acc*100), epoch)
    writer.add_scalar('Accuracy/test', int(test_acc*100), epoch)
    #writer.add_scalar('AUC/train', train_allMetrics, epoch)
    #writer.add_scalar('AUC/valid',int(valid_auc*100), epoch)
    #writer.add_scalar('AUC/test', int(test_auc*100), epoch)


In [ ]:

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
writer = SummaryWriter('tensorboard/runs')
N_EPOCHS = 3


best_valid_loss = float('inf')

Epoch2=[]
train_lossList2,valid_lossList2,test_lossList2=[],[],[]
train_accList2,valid_accList2,test_accList2=[],[],[]
train_aucList2,valid_aucList2,test_aucList2=[],[],[]
torch.cuda.empty_cache()
for epoch in tqdm(range(N_EPOCHS)):




    start_time = time.time()

    train_loss, train_acc,train_allMetrics,TP_train,TN_train,FP_train,FN_train = train(model, trainIter2, optimizer, criterion)
    valid_loss, valid_acc,valid_allMetrics = evaluate(model, validIter2, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model2.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f} | All Metrics:{train_allMetrics} | TP:{TP_train} | TN: {TN_train}')

    print(f'\tRecall:{TP_train / (FN_train + TP_train):.3f} | Precision:{TP_train / (FP_train + TP_train):.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f} | Val All Metrics:{train_allMetrics}')
    model.load_state_dict(torch.load('tut1-model.pt'))
    test_loss, test_acc,test_allMetrics= evaluate(model, testIter2, criterion)
    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f} | Test All Metrics:{train_allMetrics}')
    Epoch2.append(epoch)
    train_lossList2.append(train_loss)
    valid_lossList2.append(valid_loss)
    test_lossList2.append(test_loss)
    train_accList2.append(train_acc*100)
    valid_accList2.append(valid_acc*100)
    test_accList2.append(test_acc*100)

    train_aucList2.append(train_allMetrics)
    valid_aucList2.append(valid_allMetrics)
    test_aucList2.append(test_allMetrics)
    writer.add_scalar('Loss/train', round(train_loss,1), epoch)
    writer.add_scalar('Loss/valid', round(valid_loss,1), epoch)
    writer.add_scalar('Loss/test', round(test_loss,1), epoch)
    writer.add_scalar('Accuracy/train', int(train_acc*100), epoch)
    writer.add_scalar('Accuracy/valid',int(valid_acc*100), epoch)
    writer.add_scalar('Accuracy/test', int(test_acc*100), epoch)
    #writer.add_scalar('AUC/train', train_allMetrics, epoch)
    #writer.add_scalar('AUC/valid',int(valid_auc*100), epoch)
    #writer.add_scalar('AUC/test', int(test_auc*100), epoch)


In [ ]:
dfData=pd.DataFrame()
dfData.insert(0, 'Epoch', Epoch)
dfData.insert(1, 'train_loss', train_lossList)
dfData.insert(2, 'valid_loss', valid_lossList)
dfData.insert(3, 'test_loss', test_lossList)
dfData.insert(4, 'train_acc', train_accList)
dfData.insert(5, 'valid_acc', valid_accList)
dfData.insert(6, 'test_acc', test_accList)
dfData.insert(7, 'train_auc', train_aucList)
dfData.insert(8, 'valid_auc', valid_aucList)
dfData.insert(9, 'test_auc', test_aucList)
